In [1]:
%cd ../data/ecommerce/home-depot-product-search-relevance/
!pwd

/Users/felicildaloveme/personal_projects/data/ecommerce/home-depot-product-search-relevance
/Users/felicildaloveme/personal_projects/data/ecommerce/home-depot-product-search-relevance


In [2]:
import pandas as pd 
import numpy as np 
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/felicildaloveme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Reading the data


In [3]:
data = pd.read_csv("train.csv", encoding='iso-8859-1')
print(data.shape)

(74067, 5)


# Cleaning

### Merging the attributes, description and brands

#### Adding Attributes

In [4]:
attr_df = pd.read_csv("attributes.csv", encoding='iso-8859-1')

def merge_attributes(df):
    product_uids = df['product_uid'].values
    temp = attr_df.loc[attr_df['product_uid'].isin(product_uids)].fillna('')  
    temp['name_value'] = temp['name'] + ' ' + temp['value']
    temp['combined_attr'] = temp.groupby(['product_uid'])['name_value'].transform(lambda x: ' '.join(x))
    temp = temp.drop_duplicates('product_uid')[['product_uid', 'combined_attr']]
    df = pd.merge(df, temp, on='product_uid', how='left').set_index(df.index)
    return df

In [5]:
data = merge_attributes(data)

#### Adding 'brand' attribute as a seperate feature

In [6]:
def merge_brand(df):
    product_uids = df['product_uid'].values
    temp = attr_df.loc[attr_df['product_uid'].isin(product_uids)]  
    brands = temp[temp['name']=='MFG Brand Name']
    brands_temp = brands[['product_uid','value']]
    df = pd.merge(df, brands_temp, on='product_uid', how='left').set_index(df.index)
    df.rename(columns = {'value':'brand'}, inplace = True) 
    return df

In [7]:
data = merge_brand(data)

#### Adding description to the training data 

In [8]:
desc_df = pd.read_csv('product_descriptions.csv', encoding='iso-8859-1')

def merge_description(df):
    df = pd.merge(df, desc_df, on='product_uid', how='left').set_index(df.index)
    #an extra preprocessing step is performed to seperate the concatenated words in the description. 
    df['product_description'] = df['product_description'].apply(lambda x: ' '.join(re.findall(r'[A-Z]?[^A-Z\s]+|[A-Z]+', x)))
    return df

In [9]:
data = merge_description(data)

In [10]:
data.head(3)

,id,product_uid,product_title,search_term,relevance,combined_attr,brand,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0,Bullet01 Versatile connector for various 90Â° ...,Simpson Strong-Tie,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5,Bullet01 Versatile connector for various 90Â° ...,Simpson Strong-Tie,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0,"Application Method Brush,Roller,Spray Assemble...",BEHR Premium Textured DeckOver,BEHR Premium Textured DECKOVER is an innovativ...


### Filling Null Values

**Brand**

In [11]:
data.drop('id',inplace=True, axis=1)

In [12]:
unique_brands = np.unique(data['brand'].dropna().values)
len(unique_brands)

3142

In [13]:
def first_n(n, sent):
    if n > len(sent.split()):
        return 'error101'
    return ' '.join(sent.split()[:n])

def fillna_brand(data, unique_brnds):
    null_df = data[data['brand'].isnull()].copy()
    notnull_df = data.dropna()

    for i, row in null_df.iterrows():
        title = row['product_title']
    if first_n(4, title) in unique_brnds:
        null_df.loc[i, 'brand'] = first_n(4, title)
    elif first_n(3, title) in unique_brnds:
        null_df.loc[i, 'brand'] = first_n(3, title)
    elif first_n(2, title) in unique_brnds:
        null_df.loc[i,'brand'] = first_n(2, title)
    else:
        null_df.loc[i, 'brand'] = first_n(1, title)

    data.loc[null_df.index, 'brand'] = null_df['brand'].values
    return data

In [14]:
data = fillna_brand(data, unique_brands)

**Attributes**

* description data has no null values so we can always fill the attributes null values with the description ones.

In [15]:
def fillna_attributes(data):
    null_df = data[data['combined_attr'].isnull()].copy()
    null_df['combined_attr'] = null_df['product_description'].copy()
    data.loc[null_df.index, 'combined_attr'] = null_df['combined_attr'].values
    return data

In [16]:
data = fillna_attributes(data)

**Filling any other null values**

In [17]:
data = data.fillna('')

### Basic Preprocessing 

In [18]:
def standardize_units(text):
    text = " " + text + " "
    text = re.sub('( gal | gals | galon )',' gallon ',text)
    text = re.sub('( ft | fts | feets | foot | foots )',' feet ',text)
    text = re.sub('( squares | sq )',' square ',text)
    text = re.sub('( lb | lbs | pounds )',' pound ',text)
    text = re.sub('( oz | ozs | ounces | ounc )',' ounce ',text)
    text = re.sub('( yds | yd | yards )',' yard ',text)
    return text

def preprocessing(sent):
    sent = sent.replace('in.', ' inch ') #If we dont to this then 'in.' will be turned to 'in' in the next step
    words = re.split(r'\W+', sent)
    words = [word.lower() for word in words]
    res = re.sub("[A-Za-z]+", lambda ele: " " + ele[0] + " ", ' '.join(words)) #add space between number and alphabets in a string
    cleaned = standardize_units(res) 
    cleaned = ' '.join(cleaned.split()) #removing extra whitespaces
    return cleaned

def preprocessing_search(sent):
    sent = sent.replace('in.', ' inch ')
    words = re.split(r'\W+', sent)
    words = [word.lower() for word in words]
    res = re.sub("[A-Za-z]+", lambda ele: " " + ele[0] + " ", ' '.join(words)) #add space between number and alphabets in a string
    res = standardize_units(res) 
    res = res.replace(' in ', ' inch ') #in search_terms 'in' is used more for 'inch' than as a preposition hence this step shouldn't hurt
    cleaned = ' '.join(res.split()) #removing extra whitespaces
    return cleaned

In [19]:
data['cleaned_title'] = data['product_title'].apply(lambda x : preprocessing(x))
data['cleaned_brand'] = data['brand'].apply(lambda x : preprocessing(x))
data['cleaned_description'] = data['product_description'].apply(lambda x : preprocessing(x))
data['cleaned_attributes'] = data['combined_attr'].apply(lambda x : preprocessing(x))
data['cleaned_search'] = data['search_term'].apply(lambda x : preprocessing_search(x))

### Correcting Search Term

In [20]:
corpus = data['cleaned_title'] + " "  + data['cleaned_brand'] + " " + data['cleaned_description'] + " " + data['cleaned_attributes'] 

#removing stopwords
stp_wrds = set(stopwords.words('english'))
def stop_word_removal(sent):
    words = sent.split()
    words = [w for w in words if not w in stp_wrds]
    return ' '.join(words)

corpus = corpus.apply(lambda x : stop_word_removal(x)) 

In [21]:
#http://norvig.com/spell-correct.html
def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(' '.join(corpus.tolist())))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or set([word]))
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def corrected_term(term):
    temp = term.lower().split()
    temp = [correction(word) for word in temp]
    return ' '.join(temp)

In [22]:
data['corrected_search'] = data['cleaned_search'].apply(lambda x: corrected_term(x))

**cleaned_df2 with no stemming**

In [23]:
#stop word removal only - no stemming
def futher_preprocessing_without_stem(sent):
    sent = sent.replace('_', ' _ ')
    words = sent.split()
    words = [w for w in words if not w in stp_wrds]
    return ' '.join(words)

In [26]:
data['cleaned_title'] = data['cleaned_title'].apply(lambda x : futher_preprocessing_without_stem(x)) 
data['cleaned_brand'] = data['cleaned_brand'].apply(lambda x : futher_preprocessing_without_stem(x)) 
data['cleaned_description'] = data['cleaned_description'].apply(lambda x : futher_preprocessing_without_stem(x))
data['cleaned_attributes'] = data['cleaned_attributes'].apply(lambda x : futher_preprocessing_without_stem(x)) 
data['cleaned_search'] = data['cleaned_search'].apply(lambda x : futher_preprocessing_without_stem(x)) 
data['corrected_search'] = data['corrected_search'].apply(lambda x : futher_preprocessing_without_stem(x))
data['cleaned_attributes'] = data['cleaned_attributes'].apply(lambda x: re.sub('bullet \d\d ', '', x))
data['cleaned_description'] = data['cleaned_description'].apply(lambda x: re.sub('bullet \d\d ', '', x))
data.head()

,product_uid,product_title,search_term,relevance,combined_attr,brand,product_description,cleaned_title,cleaned_brand,cleaned_description,cleaned_attributes,cleaned_search,corrected_search
0,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,Bullet01 Versatile connector for various 90Â° ...,Simpson Strong-Tie,"Not only do angles make joints stronger, they ...",simpson strong tie 12 gauge angle,simpson strong tie,angles make joints stronger also provide consi...,versatile connector various 90â connections ho...,angle bracket,angle bracket
1,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,Bullet01 Versatile connector for various 90Â° ...,Simpson Strong-Tie,"Not only do angles make joints stronger, they ...",simpson strong tie 12 gauge angle,simpson strong tie,angles make joints stronger also provide consi...,versatile connector various 90â connections ho...,l bracket,l bracket
2,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,"Application Method Brush,Roller,Spray Assemble...",BEHR Premium Textured DeckOver,BEHR Premium Textured DECKOVER is an innovativ...,behr premium textured deckover 1 gallon sc 141...,behr premium textured deckover,behr premium textured deckover innovative soli...,application method brush roller spray assemble...,deck,deck oven
3,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Bath Faucet Type Combo Tub and Shower Built-in...,Delta,Update your bathroom with the Delta Vero Singl...,delta vero 1 handle shower faucet trim kit chr...,delta,update bathroom delta vero single handle showe...,bath faucet type combo tub shower built water ...,rain shower head,rain shower head
4,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Bath Faucet Type Combo Tub and Shower Built-in...,Delta,Update your bathroom with the Delta Vero Singl...,delta vero 1 handle shower faucet trim kit chr...,delta,update bathroom delta vero single handle showe...,bath faucet type combo tub shower built water ...,shower faucet,shower oily faucet


In [139]:
data.to_pickle('./cleaned_train_data.pkl')